## Import libraries

In [1]:
import pandas as pd
from pymongo import MongoClient
from src.GeoQueryMongo import *
from src.geocode import *
import os
from dotenv import load_dotenv
import requests

## Connect with MongoDB

As expained in the `README.md` before running `main.ipynb` you should run first the following python files in folder `scripts` in this order:

- Run `offices_processed.py` --> Obtain `companies_processed.json` 
- Run `offices_mining.py` --> Obtain `companies_mining.json`       
- Run `offices_filtered.py` --> Obtain `companies_selected.json`  

Once you load `companies_selected.json` in MongoDB with this instruction: `mongoimport --db companies --collection companies_processed --file companies_processed.json --jsonArray` you will be able to run the cells below.

In [2]:
client = MongoClient(f"mongodb://localhost/companies")
db = client.get_database()
data = list(db.companies_selected.find({}))
df=pd.DataFrame(data)

In [3]:
df.head(3)

,_id,name,category_code,office,clean_state,latitude,longitude,Starbucks,Starbucks_state,lat_s,...,lat_p,lng_p,Disco,Disco_state,lat_d,lng_d,Airport,Airport_state,lat_a,lng_a
0,5e9c7638d07dd4923741d833,Kyte,games_video,"{'type': 'Point', 'coordinates': [-122.409173,...",success,37.788482,-122.409173,"{'type': 'Point', 'coordinates': [-122.4042684...",success,37.792814,...,37.792992,-122.407870,"{'type': 'Point', 'coordinates': [-122.4102387...",success,37.791749,-122.410239,"{'type': 'Point', 'coordinates': [-122.3789554...",success,37.621313,-122.378955
1,5e9c7638d07dd4923741d834,Babelgum,games_video,"{'type': 'Point', 'coordinates': [-6.267494, 5...",success,53.344104,-6.267494,"{'type': 'Point', 'coordinates': [-6.2612559, ...",success,53.344475,...,53.347773,-6.258879,"{'type': 'Point', 'coordinates': [-6.2675772, ...",success,53.344807,-6.267577,"{'type': 'Point', 'coordinates': [-6.2499098, ...",success,53.426448,-6.249910
2,5e9c7638d07dd4923741d835,Livestream,games_video,"{'type': 'Point', 'coordinates': [-73.995625, ...",success,40.726155,-73.995625,"{'type': 'Point', 'coordinates': [-73.995485, ...",success,40.727212,...,40.728831,-73.999507,"{'type': 'Point', 'coordinates': [-73.9914785,...",success,40.726018,-73.991478,"{'type': 'Point', 'coordinates': [-73.8739659,...",success,40.776927,-73.873966


## Prepare dataset for Tableau

After trying different visualization options with Tableau (dual axis, joints...) for the dataframe `df`, the simplest way to make the representation seemed to be with a single dataset of 3 columns **latitude**, **longitude** and **names**. The cell below generates such simplified dataset.

In [4]:
#Create subset and rename columns
df_airport=df[['lat_a','lng_a','name']]
df_airport.columns=[['latitude','longitude','name']]
df_discos=df[['lat_d','lng_d','name']]
df_discos.columns=[['latitude','longitude','name']]
df_preschool=df[['lat_p','lng_p','name']]
df_preschool.columns=[['latitude','longitude','name']]
df_starbucks=df[['lat_s','lng_s','name']]
df_starbucks.columns=[['latitude','longitude','name']]
df_companies=df[['latitude','longitude','name']]
df_companies.columns=[['latitude','longitude','name']]

In [5]:
#Concatenate
objs=[df_companies, df_airport, df_discos, df_preschool, df_starbucks]
df_tableau = pd.concat(objs, axis=0)
df_tableau.reset_index(drop=True, inplace=True)

In [6]:
#Save in the output folder
df_tableau.to_json("./output/df_tableau.json",orient="records")

## Obtain map in Tableau

A file from Tablau is provided in the `output` folder: `GeoSpatial Data Project.twbx`. You can open this file with tableau and explore yourself the possible locations for the company worldwide.

<img src="output/Tableau_interface.png">

As you can see there are groups of 4 points in the same color. They are catherogized by the name of the `company`. If you zoom in tableau you will se that below the pont there is a Starbucks, Night Club, etc. An example is provided below:

<img src="output/Tableau_interface_2.png">

## Crunch API

Now we are going to use [Crunch API](https://data.crunchbase.com/reference#categories) to obtain locations for UX-Design companies. This API requires token authentification, so you will have to get one first.

In [7]:
#Load ApiKey
load_dotenv()
apiKey=os.getenv("crunch")
print("We have apiKey") if apiKey else print("No apiKey")

We have apiKey


The next stape is to make the query for the get request:

In [8]:
url = "https://api.crunchbase.com/v3.1/odm-organizations"

querystring = {"categories":"ux design",
               "organization_types":"company",
               "locations":"Toronto",
               "sort_order":"created_at DESC",
               "page":"1",
               "user_key":apiKey}

response = requests.request("GET", url, params=querystring, json='json')

Finally, with the response, we can build a dataframe with the `name`, `city`, `region` and `country` of the ux-company.

In [9]:
res = response.json()
df=pd.json_normalize(res['data']['items'])
df=df[['properties.name','properties.city_name','properties.region_name','properties.country_code']]
df.head()

,properties.name,properties.city_name,properties.region_name,properties.country_code
0,Switch Creative Solutions,Toronto,Ontario,Canada
1,Taskoob,Toronto,Ontario,Canada
2,PATIO Interactive,Toronto,Ontario,Canada
3,Counter Design Studio,Toronto,Ontario,Canada
4,BSI Labs,Toronto,Ontario,Canada


We can do geocoding by converting an addresses to geographic coordinates. These coordinates will be use to build a `near` request to MongoDB to check if we have one of the selected companies closeby the chosen ux-design company. 

In [12]:
GeoQueryMongo('Counter Design Studio Toronto')

<Response [200]>


,name,category_code,latitude,longitude
0,Batipi,games_video,43.643683,-79.403865


As can be seen the coordinates of **Batipi** are close to **Counter Design Studio Toronto**, fulfilling this way other condition. 